In [ ]:
import pandas as pd

In [ ]:
# Load frequency distribution of disposition and complexity per acuity
df_frequency = pd.read_csv('params/frequency/frequency_groupname.csv', dtype={'acuity': str, 'disposition': str, 'complexity': str, 'percent': float})

# Add column for retrospective cohort groupname and endgroup
df_frequency['groupname'] = df_frequency['acuity'].astype(str) + '-' + df_frequency['disposition'] + '-' + df_frequency['complexity']
df_frequency['endgroup'] = df_frequency['disposition'] + '-' + df_frequency['complexity']

dict_frequency = df_frequency.set_index('groupname').to_dict(orient='index')

In [ ]:
# Calculate relative and total frequencies for admitted and not admitted dispositions
dict_relative_frequency = {}
dict_total_frequency = {}
for acuity_val in ['1', '2', '3', '4', '5']:
    dict_relative_frequency[acuity_val] = {}
    dict_total_frequency[acuity_val] = {}
    for disp_val in ['HOME', 'ICU', 'WARD']:
        for comp_val in ['LOW', 'MODERATE', 'HIGH']:
            endgroup_val = disp_val + '-' + comp_val
            dict_relative_frequency[acuity_val][endgroup_val] = 0
            dict_total_frequency[acuity_val][endgroup_val] = 0

for acuity_val in ['1', '2', '3', '4', '5']:
    df_acuity = df_frequency[df_frequency['acuity'] == acuity_val]

    df_disposition_admitted = df_acuity[df_acuity['disposition'].isin(['WARD', 'ICU'])].copy()
    df_disposition_admitted['relative_freq'] = df_disposition_admitted['percent'] / df_disposition_admitted['percent'].sum()

    df_disposition_notadmitted = df_acuity[~df_acuity['disposition'].isin(['WARD', 'ICU'])].copy()
    df_disposition_notadmitted['relative_freq'] = df_disposition_notadmitted['percent'] / df_disposition_notadmitted['percent'].sum()

    for idx, row in df_disposition_admitted.iterrows():
        dict_relative_frequency[row['acuity']][row['endgroup']] = row['relative_freq']
        dict_total_frequency[row['acuity']][row['endgroup']] = df_disposition_admitted['percent'].sum()

    for idx, row in df_disposition_notadmitted.iterrows():
        dict_relative_frequency[row['acuity']][row['endgroup']] = row['relative_freq']
        dict_total_frequency[row['acuity']][row['endgroup']] = df_disposition_notadmitted['percent'].sum()

In [ ]:
# Increase frequency of high-complexity patients
for increase_val in [0.1, 0.2, 0.3, 0.4, 0.5]:
    acuity_row = []
    disposition_row = []
    complexity_row = []
    percent_row = []
    for acuity_val in ['1', '2', '3', '4', '5']:
        for disp_val in ['HOME', 'WARD', 'ICU']:
            for comp_val in ['LOW', 'MODERATE', 'HIGH']:
                grpname_val = acuity_val + '-' + disp_val + '-' + comp_val
                endgrp_val = disp_val + '-' + comp_val

                if grpname_val in df_frequency['groupname'].tolist():
                    if acuity_val != '5':
                        if disp_val in ['WARD', 'ICU']:
                            percent_row.append(min((dict_total_frequency[acuity_val][endgrp_val] + increase_val), 1) * dict_relative_frequency[acuity_val][endgrp_val])
                        else:
                            percent_row.append(max((dict_total_frequency[acuity_val][endgrp_val] - increase_val), 0) * dict_relative_frequency[acuity_val][endgrp_val])
                    else:
                        # acuity 5 only has ward disposition, so don't add for ICU
                        if disp_val in ['WARD']:
                            percent_row.append(min((dict_total_frequency[acuity_val][endgrp_val] + increase_val), 1) * dict_relative_frequency[acuity_val][endgrp_val])
                        else:
                            percent_row.append(max((dict_total_frequency[acuity_val][endgrp_val] - increase_val), 0) * dict_relative_frequency[acuity_val][endgrp_val])

                    acuity_row.append(acuity_val)
                    disposition_row.append(disp_val)
                    complexity_row.append(comp_val)

    df_new_frequency = pd.DataFrame.from_dict({'acuity': acuity_row, 'disposition': disposition_row, 'complexity': complexity_row, 'percent': percent_row}).sort_values(['acuity', 'disposition', 'complexity'])

    # Verify that sum of frequency distribution is equal to 1
    print('---', increase_val)
    print(df_new_frequency.groupby('acuity').percent.sum())

    # Save new frequency distribution
    df_new_frequency.to_csv(f'params/frequency/frequency_groupname_disposition_{int(increase_val*10)}.csv', index=False)

In [ ]:
# Calculate relative and total frequencies for high and non-high complexity
dict_relative_frequency = {}
dict_total_frequency = {}
for acuity_val in ['1', '2', '3', '4', '5']:
    dict_relative_frequency[acuity_val] = {}
    dict_total_frequency[acuity_val] = {}
    for disp_val in ['HOME', 'ICU', 'WARD']:
        for comp_val in ['LOW', 'MODERATE', 'HIGH']:
            endgroup_val = disp_val + '-' + comp_val
            dict_relative_frequency[acuity_val][endgroup_val] = 0
            dict_total_frequency[acuity_val][endgroup_val] = 0

for acuity_val in ['1', '2', '3', '4', '5']:
    df_acuity = df_frequency[df_frequency['acuity'] == acuity_val]

    df_complexity_high = df_acuity[df_acuity['complexity'].isin(['HIGH'])].copy()
    df_complexity_high['relative_freq'] = df_complexity_high['percent'] / df_complexity_high['percent'].sum()

    df_complexity_nonhigh = df_acuity[~df_acuity['complexity'].isin(['HIGH'])].copy()
    df_complexity_nonhigh['relative_freq'] = df_complexity_nonhigh['percent'] / df_complexity_nonhigh['percent'].sum()

    for idx, row in df_complexity_high.iterrows():
        dict_relative_frequency[row['acuity']][row['endgroup']] = row['relative_freq']
        dict_total_frequency[row['acuity']][row['endgroup']] = df_complexity_high['percent'].sum()

    for idx, row in df_complexity_nonhigh.iterrows():
        dict_relative_frequency[row['acuity']][row['endgroup']] = row['relative_freq']
        dict_total_frequency[row['acuity']][row['endgroup']] = df_complexity_nonhigh['percent'].sum()

In [ ]:
# Increase frequency of high-complexity patients
for increase_val in [0.1, 0.2, 0.3, 0.4, 0.5]:
    acuity_row = []
    disposition_row = []
    complexity_row = []
    percent_row = []
    for acuity_val in ['1', '2', '3', '4', '5']:
        for disp_val in ['HOME', 'WARD', 'ICU']:
            for comp_val in ['LOW', 'MODERATE', 'HIGH']:
                grpname_val = acuity_val + '-' + disp_val + '-' + comp_val
                endgrp_val = disp_val + '-' + comp_val

                if grpname_val in df_frequency['groupname'].tolist():
                    if acuity_val != '5':
                        if comp_val in ['HIGH']:
                            percent_row.append(min((dict_total_frequency[acuity_val][endgrp_val] + increase_val), 1) * dict_relative_frequency[acuity_val][endgrp_val])
                        else:
                            percent_row.append(max((dict_total_frequency[acuity_val][endgrp_val] - increase_val), 0) * dict_relative_frequency[acuity_val][endgrp_val])
                    else:
                        # acuity 5 does not have high complexity, so retain frequency distribution as is
                        percent_row.append(dict_total_frequency[acuity_val][endgrp_val] * dict_relative_frequency[acuity_val][endgrp_val])

                    acuity_row.append(acuity_val)
                    disposition_row.append(disp_val)
                    complexity_row.append(comp_val)

    df_new_frequency = pd.DataFrame.from_dict({'acuity': acuity_row, 'disposition': disposition_row, 'complexity': complexity_row, 'percent': percent_row}).sort_values(['acuity', 'disposition', 'complexity'])

    # Verify that sum of frequency distribution is equal to 1
    print('---', increase_val)
    print(df_new_frequency.groupby('acuity').percent.sum())

    # Save new frequency distribution
    df_new_frequency.to_csv(f'params/frequency/frequency_groupname_complexity_{int(increase_val*10)}.csv', index=False)